<a href="https://colab.research.google.com/github/Liraken/P04-Weather-Risk/blob/main/Machine_Learning/LSTM_training_noZHVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikeras
!pip install pyspark

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid, GridSearchCV, TimeSeriesSplit
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import make_scorer, r2_score
from google.colab import files
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

In [3]:
uploaded = files.upload()

Saving testing_history.csv to testing_history (2).csv


In [4]:
uploaded1 = files.upload()

Saving stripped.csv to stripped (10).csv


# Define Functions

In [5]:
#Creating custom R2 Callback for training and testing
class R2Callback(Callback):
    def __init__(self, X_train, y_train, patience=5, restore_best_weights=False,verbose=1):
        super(R2Callback, self).__init__()
        self.X_train = X_train
        self.y_train = y_train
        self.patience = patience
        self.best_r2 = -float('inf')  # Initialize best R^2 score
        self.verbose=verbose
        self.wait = 0
        self.restore_best_weights = restore_best_weights
        self.best_weights = None

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.X_train)
        r2 = r2_score(self.y_train, y_pred)
        if self.verbose>0:
          print(f"Epoch {epoch+1}, R^2: {r2}")

        # Check if current R^2 score is greater than the best R^2 score
        if r2 > self.best_r2:
            self.best_r2 = r2
            self.wait = 0
            if self.restore_best_weights:
                self.best_weights = self.model.get_weights()  # Save the best weights
        else:
            self.wait += 1  # Increment the counter

            # Check if we have reached the patience limit
            if self.wait >= self.patience:
                print(f"Stopping training as R^2 score hasn't improved for {self.patience} epochs.")
                if self.restore_best_weights:
                    print("Restoring best weights...")
                    self.model.set_weights(self.best_weights)  # Restore the best weights
                self.model.stop_training = True

    def reset(self):
        self.best_r2 = -float('inf')
        self.wait = 0
        self.best_weights = None

In [6]:
#Create r^2 calculator
def calculate_r_squared(y_true, y_pred):
    return r2_score(y_true, y_pred)

In [7]:
#Function to create timestep sequences
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [8]:
#function to add testing information to dataframe
def add_row_to_dataframe(df, model_name,num_timesteps, test_results,additional_notes):
    num_layers = len(model_name.layers)
    num_neurons = sum([layer.units for layer in model_name.layers if hasattr(layer, 'units')])
    new_row = {'Layers': num_layers, 'Neurons': num_neurons, 'Timesteps': num_timesteps,'Test_R^2':test_results,'Additional_notes':additional_notes}
    df = df.append(new_row, ignore_index=True)
    return df

#function to add additional notes for testing
def take_training_note():
    # Taking input from the user
    user_input = input("Define your changes: ")

    # Processing the input and getting the result
    notes = f'{user_input}'
    return notes

#create testing dataframe
testing_history=pd.read_csv('testing_history.csv')
testing_history=testing_history.drop(columns=['Unnamed: 0'])


# Read in data

In [9]:
#Create spark session
spark = SparkSession.builder \
    .appName("Read CSV with PySpark") \
    .getOrCreate()

In [10]:
#Read in CSV
df=spark.read.csv('stripped.csv', header=True, inferSchema=True)
df.show()

+------------------+---------------+
|   BEGIN_DATE_TIME|DAMAGE_PROPERTY|
+------------------+---------------+
|29-OCT-96 17:10:00|            0.0|
|21-OCT-96 09:00:00|            0.0|
|20-OCT-96 19:15:00|            0.0|
|23-JAN-96 04:00:00|            0.0|
|17-JAN-96 09:00:00|            0.0|
|17-JAN-96 22:50:00|        25000.0|
|18-JAN-96 08:45:00|        20000.0|
|06-JAN-96 20:00:00|        10000.0|
|16-JAN-96 20:00:00|            0.0|
|31-JAN-96 04:00:00|            0.0|
|31-JAN-96 05:00:00|            0.0|
|17-JAN-96 21:30:00|            0.0|
|18-JAN-96 02:00:00|            0.0|
|01-FEB-96 18:00:00|        20000.0|
|01-FEB-96 18:00:00|        20000.0|
|02-JAN-96 21:00:00|            0.0|
|07-JAN-96 08:00:00|            0.0|
|02-JAN-96 21:00:00|            0.0|
|17-JAN-96 13:00:00|            0.0|
|18-JAN-96 02:00:00|            0.0|
+------------------+---------------+
only showing top 20 rows



In [11]:
#Convert PySpark dataframe to pandas dataframe
df=df.toPandas()
df=df[['BEGIN_DATE_TIME','DAMAGE_PROPERTY']]
spark.stop()
df.head()

,BEGIN_DATE_TIME,DAMAGE_PROPERTY
0,29-OCT-96 17:10:00,0.0
1,21-OCT-96 09:00:00,0.0
2,20-OCT-96 19:15:00,0.0
3,23-JAN-96 04:00:00,0.0
4,17-JAN-96 09:00:00,0.0


In [12]:
#Define agg functions for grouping the dataframe
agg_functions={
    'DAMAGE_PROPERTY':'sum',
    'EVENT_COUNT':'first'
}

In [13]:
#Preprocessing the data
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].astype('datetime64[ns]')
df.sort_values(by=['BEGIN_DATE_TIME'],inplace=True)
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].dt.to_period('M')
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].astype('datetime64[ns]')
df['EVENT_COUNT']=df.groupby('BEGIN_DATE_TIME')['BEGIN_DATE_TIME'].transform('count')
df=df.groupby('BEGIN_DATE_TIME').agg(agg_functions).reset_index()
df['UNIX_TIMESTAMP'] = df['BEGIN_DATE_TIME'].map(pd.Timestamp.timestamp)
df.head()

,BEGIN_DATE_TIME,DAMAGE_PROPERTY,EVENT_COUNT,UNIX_TIMESTAMP
0,1996-01-01,830475710.0,6369,820454400.0
1,1996-02-01,531568800.0,4119,823132800.0
2,1996-03-01,175668080.0,4072,825638400.0
3,1996-04-01,723753290.0,4401,828316800.0
4,1996-05-01,542637240.0,6355,830908800.0


In [14]:
moddf=df.drop(columns=['BEGIN_DATE_TIME'])
moddf

,DAMAGE_PROPERTY,EVENT_COUNT,UNIX_TIMESTAMP
0,8.304757e+08,6369,8.204544e+08
1,5.315688e+08,4119,8.231328e+08
2,1.756681e+08,4072,8.256384e+08
3,7.237533e+08,4401,8.283168e+08
4,5.426372e+08,6355,8.309088e+08
...,...,...,...
330,1.186285e+09,13606,1.688170e+09
331,6.736821e+09,10910,1.690848e+09
332,8.629718e+08,4608,1.693526e+09
333,1.482090e+07,2461,1.696118e+09


In [15]:
#scaling data for PCA
scaler=StandardScaler()
scaleddf=scaler.fit_transform(moddf)

In [16]:
pca=PCA()
pca.fit(scaleddf)
pca_data=pca.transform(scaleddf)

In [17]:
#Printing features with explained variance
sorted_indices = np.argsort(pca.explained_variance_ratio_)[::-1]  # Sort in descending order
sorted_features = moddf.columns[sorted_indices]
sorted_variances = pca.explained_variance_ratio_[sorted_indices]
for feature_name, explained_variance in zip(sorted_features, sorted_variances):
    print(f"Feature '{feature_name}': {explained_variance:.4f}")

Feature 'DAMAGE_PROPERTY': 0.4087
Feature 'EVENT_COUNT': 0.3337
Feature 'UNIX_TIMESTAMP': 0.2576


# Preprocessing

In [18]:
#Split data into features and target
X=moddf.drop(columns=['DAMAGE_PROPERTY'])
y=moddf[['DAMAGE_PROPERTY']]

In [19]:
#Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Normalize the target
scaler_y = MinMaxScaler(feature_range=(0, 1))
# y_normalized =pd.DataFrame(scaler_y.fit_transform(y.reshape(-1,1)), columns=y.columns)
y_reshaped=y.values.reshape(-1,1)
y_normalized =pd.DataFrame(scaler_y.fit_transform(y_reshaped), columns=y.columns)

In [20]:
train_size = int(len(X_normalized) * 0.8)
val_size=int(len(X_normalized) * 0.1)
test_size = int(len(X_normalized) - train_size - val_size)


In [21]:
time_steps = 7
X_train, y_train = create_sequences(X_normalized.iloc[0:train_size], y_normalized.iloc[0:train_size], time_steps)
X_val, y_val = create_sequences(X_normalized.iloc[train_size:train_size+val_size], y_normalized.iloc[train_size:train_size+val_size], time_steps)
X_test, y_test = create_sequences(X_normalized.iloc[train_size+val_size:len(X_normalized)], y_normalized.iloc[train_size+val_size:len(X_normalized)], time_steps)

# Testing and training

In [22]:
#Define model for testing hyperparameters
def create_lstm_model(n_layers, n_neurons, n_timesteps, n_features):
    model = Sequential()
    for i in range(n_layers - 1):
      if i>0:
        model.add(LSTM(n_neurons,return_sequences=True))
        tf.keras.layers.Dropout(0.2)
    model.add(LSTM(n_neurons))
    tf.keras.layers.Dropout(0.2)
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

In [23]:
param_grid = {
    'n_layers': [2, 3, 4, 5,6],
    'n_neurons': [16,32,64,128],
    'time_steps': [11,13,15,17,19]
}

In [24]:
best_model = None
best_r2 = float(0)
r2_callback = R2Callback(X_val, y_val, patience=10, restore_best_weights=True, verbose=0)

In [25]:
#For loop to test different hyperparameters
for params in ParameterGrid(param_grid):
    print("Testing parameters:", params)
    X_train, y_train = create_sequences(X_normalized.iloc[0:train_size], y_normalized.iloc[0:train_size], time_steps)
    X_val, y_val = create_sequences(X_normalized.iloc[train_size:train_size+val_size], y_normalized.iloc[train_size:train_size+val_size], time_steps)
    X_test, y_test = create_sequences(X_normalized.iloc[train_size+val_size:len(X_normalized)], y_normalized.iloc[train_size+val_size:len(X_normalized)], time_steps)
    model = create_lstm_model(params['n_layers'], params['n_neurons'], params['time_steps'], len(X_normalized.columns))
    r2_callback.reset()
    model.fit(X_train, y_train, epochs=100000, verbose=0,callbacks=[r2_callback])
    r_squared=calculate_r_squared(y_test, model.predict(X_test))
    print("R^2 value:",r_squared)
    if r_squared > best_r2:
        best_r2 = r_squared
        best_model = model

print(f"Best model parameters:{params['n_layers'], params['n_neurons'], params['time_steps']}")
print(best_model.summary())
print(f"Best R^2 value: {best_r2}")

Testing parameters: {'n_layers': 2, 'n_neurons': 2, 'time_steps': 11}
1/1 [==============================] - 0s 16ms/step
Stopping training as R^2 score hasn't improved for 10 epochs.
Restoring best weights...
1/1 [==============================] - 0s 16ms/step
R^2 value: -0.17267675832007057
Testing parameters: {'n_layers': 2, 'n_neurons': 2, 'time_steps': 13}
1/1 [==============================] - 0s 15ms/step
Stopping training as R^2 score hasn't improved for 10 epochs.
Restoring best weights...
1/1 [==============================] - 0s 15ms/step
R^2 value: -0.14387688581629798
Testing parameters: {'n_layers': 2, 'n_neurons': 2, 'time_steps': 15}
1/1 [==============================] - 0s 16ms/step
Stopping training as R^2 score hasn't improved for 10 epochs.
Restoring best weights...
1/1 [==============================] - 0s 18ms/step
R^2 value: 0.011682323867642452
Testing parameters: {'n_layers': 2, 'n_neurons': 2, 'time_steps': 17}
1/1 [==============================] - 0s 16ms/s

In [41]:
time_steps = 13
X_train, y_train = create_sequences(X_normalized.iloc[0:train_size], y_normalized.iloc[0:train_size], time_steps)
X_val, y_val = create_sequences(X_normalized.iloc[train_size:train_size+val_size], y_normalized.iloc[train_size:train_size+val_size], time_steps)
X_test, y_test = create_sequences(X_normalized.iloc[train_size+val_size:len(X_normalized)], y_normalized.iloc[train_size+val_size:len(X_normalized)], time_steps)

In [42]:
test_model.reset_states()
test_model= Sequential()
test_model.add(LSTM(units=16,return_sequences=False))
tf.keras.layers.Dropout(0.2)
# test_model.add(LSTM(units=4))
test_model.add(Dense(units=1))

test_model.compile(optimizer=Adam(learning_rate=0.00001), loss='mse')
r2_callback.reset()

r2_callback = R2Callback(X_val, y_val, patience=15, restore_best_weights=True, verbose=1)

In [43]:
training_notes=take_training_note()

Define your changes: attempting to validate best hyperparameters


In [44]:
test_model.fit(X_train, y_train, epochs=100000, verbose=1,callbacks=[r2_callback])
r_squared=calculate_r_squared(y_test, test_model.predict(X_test))
testing_history=add_row_to_dataframe(testing_history,test_model,time_steps,r_squared,training_notes)
testing_history

Epoch 1/100000
1/1 [==============================] - 0s 297ms/step
Epoch 1, R^2: -1.0623070191057944
8/8 [==============================] - 2s 52ms/step - loss: 0.0075
Epoch 2/100000
1/1 [==============================] - 0s 17ms/step
Epoch 2, R^2: -0.9981768208088115
8/8 [==============================] - 0s 12ms/step - loss: 0.0074
Epoch 3/100000
1/1 [==============================] - 0s 16ms/step
Epoch 3, R^2: -0.9410008403839367
8/8 [==============================] - 0s 12ms/step - loss: 0.0074
Epoch 4/100000
1/1 [==============================] - 0s 16ms/step
Epoch 4, R^2: -0.8813360604103746
8/8 [==============================] - 0s 11ms/step - loss: 0.0074
Epoch 5/100000
1/1 [==============================] - 0s 17ms/step
Epoch 5, R^2: -0.8285371440320433
8/8 [==============================] - 0s 12ms/step - loss: 0.0073
Epoch 6/100000
1/1 [==============================] - 0s 16ms/step
Epoch 6, R^2: -0.7751528568743995
8/8 [==============================] - 0s 12ms/step - loss

<ipython-input-8-b729cd278139>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


,Layers,Neurons,Timesteps,Test_R^2,Additional_notes
0,2,9,5,0.041060,NaN
1,2,9,5,0.037342,increased learning rate from 0.00001 to 0.0001
2,2,9,5,-0.166013,increased learning rate from 0.00001 to 0.0001
3,2,9,5,-0.035526,added 0.2 dropout layer
4,2,9,5,-0.010439,increased dropout to 0.4
5,2,9,5,-0.220662,reduced learning rate to 0.00001
6,3,9,5,-0.083424,reduced learning rate to 0.00001
7,2,9,5,-0.259303,testing best model
8,2,9,5,-0.017739,testing best model
9,2,5,5,0.024612,NaN


In [48]:
files.download('testing_history.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>